## Getting Started With Deep Learning

Collaboratively Built by Microsoft Corporation and NVidia Corporation

### Overview

Deep learning is allowing machines to acheive near human levels of visual recognition capabilities and disrupting many applications by replacing hand-coded software with predictive models learned directly from data.  This lab introduces the machine learning workflow and provides hands-on experience with using deep neural networks (DNN) to solve a challenging real-world image classification problem.  We will work through all phases of the deep learning workflow including data preprocessing, training, evaluation, and methods to improve training accuracy including data augmentation and network optimization. You will also see the benefits of GPU acceleration in the model training process.  On completion of this lab you will have the knowledge to train a DNN on your own image classification dataset.



### Lab Lessons

1.  What is deep learning and convolutional networks?
2.  How to create a database
3.  How to create label files for classification
4.  Exploring the data set
5.  Create a classification model
6.  Understanding the results
7.  Moving forward to object detection

## Introduction to Deep Learning

Deep learning is the use of many hidden layers in an artificial neural network to train a model to learn and understand the data without an inherent understanding of what the data is.  Deep learning is being used by many different disciplines to allow computers to learn from vast amounts of data.  Recent advances in computer vision, object detection and natural language processing can be attributed to the adoption of Deep Learning techniques.

One key to the success of deep learning is the convolutional neural network (CNN).  In a traditional neural network, a artificial neurons in a layer are fully connected to the previous and next layer.  In a [convolutional neural network](http://en.wikipedia.org/wiki/Convolutional_neural_network), overlapping regions of the network are associated instead of the entire layer.  Biologically inspired, a convolutional network acts as filters to process pieces of data.  The result is that these networks can act as feature extractors at both coarse and fine scales eliminating the need to design custom feature extractors as was done previously in traditional computer vision.  CNNs can be trained to recognize structure in large sets of data including images, voice, and text.   

The workflow of deep learning is a multi-phase, iterative process.  
    1.  First, data must be gathered and pre-processed.  While the Internet and big data have provide access to massive quantities of data, the data often needs to be verified, labeled, and pre-processed for consistency.  
    2.  Second, the data is used to train a network.  Pre-existing networks can be used or new ones can be developed.  Both the network and the training process have many variables that can be modified and tuned which affect the training rate and accuracy.
    3.  Third, models need to be tested to verify they are working as expected.  Often at this point one iteriates over the steps to improve the results.  This includes Reprocessing data, modifying networks, modifying parameters of the networks or solvers, and retesting until the desired results are obtained.

In this tutorial today, you will work through all phases of the deep learning workflow to solve a classic machine learning problem, image classification.  We will be using a multi-layer convolutional neural network to train our model to recognize several types of images from the Microsoft Coco database. The Coco database contains hundreds of thousands of images with millions of various objects contained within them.  Coco is a classic data set for various computer vision tasks.  We will learn to classify an image between 11 different classes.

To simplify the process of the deep learning workflow, we will be using NVIDIA's [DIGITS](https://developer.nvidia.com/digits).  DIGITS is a Deep Learning GPU Training System that helps users develop and test convolutional neural networks. DIGITS supports multiple frameworks, data formats, and training goals including image classification and object detection.

In addition, DIGITS also includes a workload manager.  The workload manager allows the user to launch multiple jobs of different types and it coordinates access to the local resources.  If multiple GPUs are present, jobs can run simultaneously.  If more jobs are created than the resources available, the jobs will be queued until resources become free.  The DIGITS dashboard allows the user to monitor all of the active jobs and their current state.

We will be using the [Caffe](http://caffe.berkelyvision.org) framework.  Caffe was created by the Berkeley Vision and Learning Center (BLVC).  Caffe is a flexible and extensible framework that provides researchers the means to train networks without writing all of the code necessary to do so.  Model training can be parallelized across multiple GPUs to accelerate learning.

To decrease training time, simplify the provisioning process as well as reduce costs, we will be using the Azure N Series virtual machines; which provide hardware access to 1-4 K-80 GPUs depending on configuration.

This tutorial will cover the typical tasks of the deep learning workflow.  First, we will review the creation of the Coco data set for classification.  Second, we will train a model to classify the Coco images.  Third, we will test the trained model against other test data and analyze the results.  After this, we will augment the data and modify the standard network to try and improve our image classification accuracy.

## Starting DIGITS

When you start DIGITS, you will be taken to the home screen.

![](images/digits_home.png)

You can create new datasets or new models.  This home page will show all of your currently processing and completed models and datasets.  The default window pane shows the models.  If you wish to look at the datasets, select the **Datasets** tab on the left.  On the right there are two tabs **New Datasets** and **New Models**.  For now we are going to create a new **Dataset**.  You can choose either **Classification**, **Object Detection** or **Other**.  In this tutorial, we will be using the **Classification** option.

![](images/digits_zoom_new_dataset.png)

## Review - Creating a Database
To create the Coco dataset, we had to first download the coco database to the virtual machine using the following shell commands.  **The data set already exists on this vm, so executing the below lines will cause out of disk errors.**

In [1]:
#Below will take a while to download
#Ensure you have enough disk, 100GB free space should do.
#Uncomment and execute should you have the free disk space, otherwise you will run out of disk.

#Get Data
#!wget http://msvocds.blob.core.windows.net/coco2014/train2014.zip
#!wget http://msvocds.blob.core.windows.net/coco2015/test2015.zip

#Get Labels
#!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/instances_train-val2014.zip
#!wget http://msvocds.blob.core.windows.net/annotations-1-0-4/image_info_test2015.zip

#Install Unzip
#!sudo apt-get install unzip

#Unzip data and labels
#!unzip train2014.zip
#!unzip test2015.zip
#!unzip instances_train-val2014.zip
#!unzip image_info_test2015.zip

The next stage is to create two white space seperated text files describing the locations of the test and validation data along side a zero indexed class label and a master label file.  The sample segment of the classification description file can be found below.  Notice how the file name includes only numbers.  This is a requirement of digits.

/home/drcrook/db/coco/train2014/000000355674.jpg 8

/home/drcrook/db/coco/train2014/000000355674.jpg 8

/home/drcrook/db/coco/train2014/000000062929.jpg 8

/home/drcrook/db/coco/train2014/000000487555.jpg 7

/home/drcrook/db/coco/train2014/000000434381.jpg 7

/home/drcrook/db/coco/train2014/000000183734.jpg 7

/home/drcrook/db/coco/train2014/000000400593.jpg 7

The master label file will be of the form as seen below, where each line corresponds to the zero indexed label in the classification description file.  You can think of a master label file as a lookup dictionary between the classification description file's labels and the actual text representation of that label.

person
bicycle
car
motorcycle
airplane
bus
train
truck
boat
trafficlight
firehydrant

The fastest way to actually perform this task is to write some quick python code to generate these files.  You will find that the Coco dataset's annotation files simply contain all of the data, but is not properly formatted for digits.  This is a common issue regardless of deep learning platform or dataset.  Should you decide to collect your own data set, you will need to generate the above files as well.

We have included some python code to help get you started.

In [ ]:
#%% imports
import pandas as pd
import json
import os

#%% Generate actual file names as they exist on disk for label file
def AddFileLocations(ann_df):
    file_locations = []
    for file in ann_df['image_id']:
        num_z = 16 - len(file + '.jpg')
        prefix = ''
        for i in range(0, num_z):
            prefix += '0'
        file_locations.append(prefix + file + '.jpg')
    ann_df['Path'] = file_locations
    return ann_df


#%% Extract category name as category id is not in order or zero indexed.
def AddCategoryNames(ann_df):
    cat_names = []
    for item in ann_df['category_id']:
        cat_names.append(ann_cats[ann_cats['id'] == item]['name'].tolist()[0].replace(' ', ''))
    ann_df['cat_name'] = cat_names
    return ann_df

#%% Read Data
def ReadData(path):
    raw_json_train = json.load(open(path, 'r'))
    #get annotations as a data frame
    ann_df = pd.io.json.json_normalize(raw_json['annotations'])
    #img id is actually the file name without the prefix.
    ann_df['image_id'] = ann_df['image_id'].astype('str')
    ann_df = AddFileLocations(ann_df)
    ann_df = AddCategoryNames(ann_df)
    #zero index the category id
    ann_df['category_id'] = ann_df['category_id'] - 1
    return ann_df

#%% File Location Definitions
ann_base_dir = 'YOUR BASE DIRECTORY'
data_base_dir = 'YOUR BASE DIR'
train_lbls = 'instances_train2014.json'
test_lbls = 'instances_val2014.json'

#%% Read Data 
#Coco is in json format
train_df = ReadData(ann_base_dir + train_lbls)
test_df = ReadData(ann_base_dir + test_labls)
#Use data base dir
train_df['Path'] = data_base_dir + train_df['Path']
test_df['Path'] = data_base_dir + test_df['Path']
#get categories as a data frame.
ann_cats = pd.io.json.json_normalize(raw_json['categories'])

#%% Save File
base_label_dir = 'YOUR BASE DIR'
ann_cats['name'].to_csv(base_label_dir + 'master_label_file.txt', sep = ' ', index = False, header = False)
train_df[['Path', 'category_id']].to_csv(base_label_dir + 'train_classify_coco.txt', sep = ' ', index = False, header = False)
test_df[['Path', 'category_id']].to_csv(base_label_dir + 'test_classify_coco.txt', sep = ' ', index = False, header = False)

Now that we have downloaded some data and have the proper description files, we can create a database. To create a database, we selected **Classification** from the **New Dataset** menu.    At this point you may need to enter a username.  If requested, just enter any name in lower-case.

In the **New Dataset** window, ensure you have selected **Use Text Files**.  You will want to set the following fields to the values specified:

- Image Type : Color
- Image Size : 256 x 256
- Training: YOURPATH/train_classify_coco.txt
- Validation: YOURPATH/test_classify_coco.txt
- Labels : YOURPATH/master_label_file.txt
- Group Name : classify
- Dataset Name : coco_classify

Your screen should look like the image below.

![](images/digits_new_classification_dataset.png)

When you have filled in the fields, press the **Create** button.

The next window will show you the progress of the job and the estimated time to completion.  This may take some time.

When it is done, you can explore the database.  Find the **Explore** Button at the bottom of the **Create DB (train)** panel.  

![](images/digits_explore_db.png)

Here you can scan through all of the images in the database. When the database is created, the image order is randomized. Models will train faster and more accuractely when they do not process the images in order (process all of the images of zeros, then all of the images of ones, etc.). When you explore your database, your database will be in a different order than the one shown here. 

On this page you can see several images from the coco dataset with labels.  How many problems can you identify that we will encounter with this data set performing image classification?  Can you think of ways to get around some of these potential issues?  Many of the issues you find are very common problems in this field.  In the second section of this workshop we will cover a very successful strategy, but for now, lets continue anyways.

## Task 2 - Create the Model
Now that we have a database of images, lets train our network.  At the top right of every page, the name of our training system, **DIGITS**, is visible.  If you click the name it will take you back to your home page.  From here, we can select **Classification** from the **New Model** menu.  

In the **New Image Classification Model** page, there are many options available to configure and tune your training session.  Some of the more typically used ones are:  

- **Select Dataset** - Choose one of your databases to use for training.
- **Training Epochs** - Select the number of epochs for training.  An epoch is one iteration through the training data.  The number of epochs to use depends on the data and the model, but can be as few as 5 or over 100.
- **Snapshot Interval** - The frequency, in epochs, that the model state is saved to a file.
- **Validation Interval** - The frequency, in epochs, that the accuracy of the model is computed against the validation data.  This is important so you can monitor the progress of your training sessions.
- **Random Seed** - Specifies the value of seed should be used by the random number generator.  By setting this value, then the initial model will be randomize to the same state for different training sessions.
- **Batch Size** - The batch size is the number of images to use at one time.  The larger the batch size, the more parallelism that can be acheived and the faster the training will progress.  The batch size will be constrained by the size of available memory in your GPU.  You typically want to use the largest value possible.
- **Base Learning Rate** - This value specifies at what rate the network will learn. The weights of the model are found using some gradient descent method.  The value describes the size of the step to be taken for each iteration.  Too large of a value and the weights will change to quickly and the model may not converge.  Too small of a value and the solution will take longer to converge. 

DIGITS currently has built-in support for three networks.  [LeNet](http://yann.lecun.com/exdb/lenet/) is a convolutional network originally developed to recognize hand written digits. In 2012 [AlexNet](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf) won the [ImageNet](http://image-net.org/) compitition using a Deep Neural network instead of traditional computer vision techniques. This revolutionized the field of computer vision and within a couple of years all of the top entries in the ImageNet competition were based on Deep Neural Networks.  [GoogleNet](https://arxiv.org/abs/1409.4842) in 2014 set a new standard of image classification in the ImageNet competition.

DIGITS also supports two frameworks.  Caffe is the one with which we will be working today.  Torch (http://torch.ch/) is another framework that is good at image classification as well as speech recognition and language processing and.

To train our model, we want to set the following options:

- Select the **coco_classify** dataset
- Set the number of **Training Epochs** to 10
- Set the framework to **Caffe**
- Set the model to **AlexNet**
- Set the name of the model to **YOURNAME_coco_classify**

![](images/digits_create_new_model.png)

When you have set all of these options, press the Create button.  You are now training your model!  For this configuration, the model training should complete in less than a minute.  While the training progresses, the statistics of the model are updated in the window.  The chart in the middle provides key information to how well your training is doing.

![](images/digits_model_loss_accuracy.png)

Three quantities are reported training loss, validation loss, and accuracy.  The values of training and validation loss should decrease from epoch to epoch, although they may jump around some.  The accuracy is the measure of the ability of the model to correctly classify the validation data.  If you hover your mouse over any of the data points, you will see its exact value.  In this case, the accuracy at the last epoch is about 70%.  Your results might be slightly different that what is shown here since the initial networks are generated randomly.

An accuracy of 70% is not nearly cutting edge, perhaps we can test some images or explore the database to find the specific reasons for this?

### Test The Model

Lets test the ability of the model to identify other images.  If you go to the bottom of the window, you can test a single image or a list of images.  On the left, type in the path **/home/drcrook/db/coco/val2014/000000005535.jpg** in the Image Path text box.  Select the **Show visualizations and statistics** checkbox, then select the **Classify One** button.  After a few seconds, a new window is displayed with the image and information about its attempt to classify the image.

![](images/digits_classify_one_1.png)
![](images/digits_classify_one_2.png)

The figure above show the top portion of what you will see in your window.  The data in this window provides information about how well the model is doing but it also provides information regarding what each of the layers is doing. You can see that the model reported that there is a 78.22% chance that your image is of a person, but notice 6.25% being a car, 3.16% being a truck or 2.9% being a bus.  As a human, can you decide if this picture is of the person, or a vehicle?

The first layer scales the data.  The scaled and data layer shows how the original image is scaled and what the resulting image looks like.  The conv1 layer extracts the most basic features out of the image using a series of two dimensional filters.  You can see that in the conv1 Weights section there are a series of square images which represent the weights for each filter (kernel).  Each of these these kernels has learned something about the low level features of the images.  These kernels slide over the entire image and begin to activate.  This is reported in the conv1 Activiation section where the data output is reported as **[96 55 55]**.  As data are processed through the CNN, the images continue to get smaller which allows the network to detect different features.  

At the bottom of this page (not shown here), the statistics page reports the **Total Learned Parameters** as 56,913,291.  This is exactly why we use GPU accelerated compute.

Lets try a few more images to see how the performance is.
/home/drcrook/db/coco/val2014/000000027390.jpg
/home/drcrook/db/coco/val2014/000000028114.jpg
/home/drcrook/db/coco/val2014/000000030044.jpg
/home/drcrook/db/coco/val2014/000000031092.jpg

Not particularly impressive is it?  We can see here that accuracy is not a good representation.  What we have is a few key issues.  

First, we have a very **heavily skewed data set.**
![](images/digits_skewed_data.png)

Second, we have many of the same image being classified for various classes as it contains people, buses and even bicycles all in the same picture.
![](images/digits_bus.png)
![](images/digits_person.png)
What we can gather from this is that the folks labelling the data were capturing ALL items in a picture, but not necessarily the primary point of the image.  If you inspect the coco dataset further, you will notice there are annotations for bounding boxes for each class and each image is referenced many many times for each of those items.

## Next Steps

In our example here, we struggled to identify holistic images, but we have the data we need to localize all objects within a scene.  This scenario, known as object detection is a bit more useful in robotics scenarios as you can localize specific objects within a camera frame and actuate LEDs and motors from this type of information; which is exactly what we will do during the second half of the day using the model we build in the next workshop.

## Summary

In this tutorial you were provided an introduction to the Deep Learning and all of the steps necessary to classify images including data processing, training, testing, as well as very common pitfalls that you will encounter as well as insights into what you must do to resolve those pitfalls.  Perhaps now you can think of new ways to adjust either your data, labels or techniques to acheive the results you might be looking for.

